# Saving and restoring variables in Tensorflow

If you dont pass arguments to tf.train.Saver(), saver handler saves all variables in graph
each variable is saved under the name that was passed when the variable was created

may be useful to only restore a subset of variables used by any model.

In [30]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
from edward.models import Categorical, Normal
import edward as ed
import pandas as pd

In [31]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


For example, you may have trained a neural net with five layers, and you now want to train a new model with six layers that reuses the existing weights of the five trained layers. 

In [32]:
tf.reset_default_graph()

#Placeholders
x = tf.placeholder(tf.float32,[None,784])
y_ph = tf.placeholder(tf.float32,[None,10])

w1 = tf.Variable(tf.random_normal([784,256]), name="w1")
b1 = tf.Variable(tf.random_normal([256]),name="b1")

w2 = tf.Variable(tf.random_normal([256,10]), name="w2")
b2 = tf.Variable(tf.random_normal([10]),name="b2")

In [33]:
def neural_netork(x,w1,w2,b1,b2):
    h = tf.nn.relu(tf.matmul(x,w1)+b1)
    y = tf.nn.relu(tf.matmul(h,w2)+b2)
    return y
logits = neural_netork(x,w1,w2,b1,b2)

In [34]:
# define cost
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=logits,labels=y_ph)
#define loss
loss_operation = tf.reduce_mean(cross_entropy)
# define optimizer
training_operation = tf.train.AdamOptimizer(.001).minimize(loss_operation)



In [35]:
# Define how accuracy will be computed
correct_pred = tf.equal(tf.argmax(logits,1), tf.argmax(y_ph,1))
acc_operation = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [36]:
sess = tf.InteractiveSession()
tf.global_variables_initializer().run()

for i in range(20000):
    x_batch, y_batch = mnist.train.next_batch(128)
    sess.run(training_operation,feed_dict={x:x_batch, y_ph:y_batch})
    if i%1000==0:
        print acc_operation.eval(feed_dict={x:x_batch,y_ph:y_batch})

#     saver = tf.train.Saver({"w1":w1,"b1":b1})
#     saver.save(sess,save_path="/tmp/model.ckpt")
#     print("Model saved in file: %s" % "/tmp/model.ckpt")

0.0546875


KeyboardInterrupt: 

In [28]:
va = w1.eval()

ba = b1.eval()

In [22]:
tf.reset_default_graph()

#Placeholders
x = tf.placeholder(tf.float32,[None,784])
y_ph = tf.placeholder(tf.float32,[None,10])

w1 = tf.get_variable(initializer=tf.constant(va), name="w1")
b1 = tf.get_variable(initializer=tf.constant(ba),name="b1")

w2 = tf.Variable(tf.random_normal([256,10]), name="w2")
b2 = tf.Variable(tf.random_normal([10]),name="b2")

ValueError: Cannot use the default session to evaluate tensor: the tensor's graph is different from the session's graph. Pass an explicit session to `eval(session=sess)`.